# Tree Methods Consulting Project 

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one!
Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.
___

**Think carefully about what this problem is really asking you to solve. While we will use Machine Learning to solve this, it won't be with your typical train/test split workflow. If this confuses you, skip ahead to the solution code along walk-through!**
____

In [2]:
import findspark

In [3]:
findspark.init('/home/coffey/spark-2.2.0-bin-hadoop2.7')

In [5]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('tree').getOrCreate()

In [8]:
df = spark.read.csv('dog_food.csv', inferSchema=True, header=True)

In [11]:
df.show(4)

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
+---+---+----+---+-------+
only showing top 4 rows



In [10]:
df.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: double (nullable = true)



#import vector assembler

In [12]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(inputCols=['A','B','C','D'], outputCol='features')

In [16]:
df1 = assembler.transform(df)

In [17]:
df1.show(4)

+---+---+----+---+-------+------------------+
|  A|  B|   C|  D|Spoiled|          features|
+---+---+----+---+-------+------------------+
|  4|  2|12.0|  3|    1.0|[4.0,2.0,12.0,3.0]|
|  5|  6|12.0|  7|    1.0|[5.0,6.0,12.0,7.0]|
|  6|  2|13.0|  6|    1.0|[6.0,2.0,13.0,6.0]|
|  4|  2|12.0|  1|    1.0|[4.0,2.0,12.0,1.0]|
+---+---+----+---+-------+------------------+
only showing top 4 rows



In [18]:
final_data = df1.select(['Spoiled', 'features'])

#1. split data, 2.train model, 3.evaluate!

In [21]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [26]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [23]:
rfc = RandomForestClassifier(labelCol='Spoiled')

In [24]:
rfc_model = rfc.fit(train_data)

In [25]:
rfc_preds = rfc_model.transform(test_data)

In [28]:
rfc_preds.show(4)

+-------+-----------------+--------------------+--------------------+----------+
|Spoiled|         features|       rawPrediction|         probability|prediction|
+-------+-----------------+--------------------+--------------------+----------+
|    0.0|[1.0,2.0,9.0,1.0]|[18.2172131147541...|[0.91086065573770...|       0.0|
|    0.0|[1.0,3.0,8.0,5.0]|[19.9444444444444...|[0.99722222222222...|       0.0|
|    0.0|[1.0,4.0,8.0,5.0]|          [20.0,0.0]|           [1.0,0.0]|       0.0|
|    0.0|[1.0,5.0,8.0,3.0]|[19.5981654957064...|[0.97990827478532...|       0.0|
+-------+-----------------+--------------------+--------------------+----------+
only showing top 4 rows



In [31]:
#evaluate!
my_eval = MulticlassClassificationEvaluator(labelCol='Spoiled', metricName='accuracy')

In [33]:
my_eval.evaluate(rfc_preds)

0.993006993006993

# Finishing practing Random Forest ML process. I want to find what preservative has the most influence on spoiling.

In [38]:
rfc_model.featureImportances

SparseVector(4, {0: 0.0309, 1: 0.0287, 2: 0.906, 3: 0.0344})

In [39]:
rfc_model_whole = rfc.fit(final_data)

In [40]:
rfc_model_whole.featureImportances

SparseVector(4, {0: 0.0216, 1: 0.0297, 2: 0.9246, 3: 0.024})

# Conclusion: The third preservative has the most influence on spoiling.